In [1]:
from utilsADCN import mnistLoader, plotPerformance
from ADCNbasic import ADCN
from ADCNmainloop import ADCNmainMT
import numpy as np
import pdb
import torch
import random
from torchvision import datasets, transforms

In [2]:
# random seed control
np.random.seed(0)
torch.manual_seed(0)
random.seed(0)

In [3]:
# convert data to torch.FloatTensor
transform = transforms.ToTensor()

# load the training and test datasets
labeledData   = datasets.MNIST(root='data', train=False,download=True, transform=transform)
unlabeledData = datasets.MNIST(root='data', train=True, download=True, transform=transform)

Failed to download (trying next):
HTTP Error 503: Service Unavailable



100%|██████████| 9912422/9912422 [00:11<00:00, 870805.93it/s] 


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 345582.62it/s]


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable



100%|██████████| 1648877/1648877 [00:00<00:00, 3355222.21it/s]


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable



100%|██████████| 4542/4542 [00:00<00:00, 952812.28it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [4]:
dataStream = mnistLoader(labeledData, unlabeledData, nEachClassSamples = 500)

Number of output:  10
Number of labeled data:  5000
Number of unlabeled data:  65000
Number of unlabeled data batch:  65


In [10]:
device = torch.device('mps')

In [6]:
dataStream.createTask(nTask = 5, taskList = [[0,1],[2,3],[4,5],[6,7],[8,9]], taskType = 3)

Number of task:  5
Number of labeled data per task:  1000
Number of unlabeled data per task:  12000
Number of unlabeled data batch per task:  12
Number of unlabeled data test:  5000


In [7]:
allMetrics = []

In [8]:
nNodeInit  = 96  # 96  # 32
epoch      = 1
n_trials   = 5

In [11]:
for i_trial in range(0, n_trials):
    print('Trial: ', i_trial)
    ADCNnet = ADCN(dataStream.nOutput, nHiddenNode = nNodeInit)
    ADCNnet.nOutputPerTask = 2
    ADCNnet, performanceHistory0, allPerformance0 = ADCNmainMT(ADCNnet, dataStream, noOfEpoch = epoch, device = device)
    allMetrics.append(allPerformance0)

Trial:  0
Network initialization phase is started
+++ Grow node +++
+++ Grow node +++
+++ Grow node +++
+++ Grow node +++
+++ Grow node +++
+++ Grow node +++
+++ Grow node +++
+++ Grow node +++
+++ Grow node +++
+++ Grow node +++
+++ Grow node +++
+++ Grow node +++
+++ Grow node +++
--- Prune node ---
--- Prune node ---
--- Prune node ---
--- Prune node ---


Cluster initialization phase is started
A cluster was created containing  473  centroids
1 -th batch 0 -th task


/Users/seanfuhrman/Desktop/Current Programs/SEELab/Master's/UCL for IDS/AutonomousDCN/ADCNbasic.py:377: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/python_arg_parser.cpp:1630.)
  loss.add_(self.regStrClusteringLoss/2,clusteringLoss(latentAE, oneHotClusters, centroids))


Accuracy:  100.0
2 -th batch 0 -th task
A cut point is detected cut:  500
H0 is NOT rejected, size: 0.06767930090427399 ; Error bound warning:  0.4806996285915375 ; Error bound drift:  0.5291503071784973
Status: STABLE
3 -th batch 0 -th task
A cut point is detected cut:  500
H0 is NOT rejected, size: 0.04266124963760376 ; Error bound warning:  0.4894787669181824 ; Error bound drift:  0.5388143658638
Status: STABLE
4 -th batch 0 -th task
A cut point is detected cut:  500
H0 is NOT rejected, size: 0.03847557306289673 ; Error bound warning:  0.49020254611968994 ; Error bound drift:  0.5396109819412231
Status: STABLE
5 -th batch 0 -th task
A cut point is detected cut:  500
H0 is NOT rejected, size: 0.04376441240310669 ; Error bound warning:  0.48371201753616333 ; Error bound drift:  0.5324663519859314
Status: STABLE
6 -th batch 0 -th task
A cut point is detected cut:  500
H0 is NOT rejected, size: 0.06325415521860123 ; Error bound warning:  0.48352012038230896 ; Error bound drift:  0.53225

In [12]:
# all results

# 0: accuracy
# 1: all tasks accuracy
# 2: BWT
# 3: FWT
# 4: ARI
# 5: NMI
# 6: f1_score
# 7: precision_score
# 8: recall_score
# 9: training_time
# 10: testingTime
# 11: nHiddenLayer
# 12: nHiddenNode
# 13: nCluster
# 14: nMemory

meanResults = np.round_(np.mean(allMetrics,0), decimals=2)
stdResults  = np.round_(np.std(allMetrics,0), decimals=2)

print('\n')
print('========== Performance ==========')
print('Preq Accuracy: ', meanResults[0].item(), '(+/-)',stdResults[0].item())
print('All tasks accuracy: ', meanResults[1].item(), '(+/-)',stdResults[1].item())
print('BWT: ', meanResults[2].item(), '(+/-)',stdResults[2].item())
print('FWT: ', meanResults[3].item(), '(+/-)',stdResults[3].item())
print('ARI: ', meanResults[4].item(), '(+/-)',stdResults[4].item())
print('NMI: ', meanResults[5].item(), '(+/-)',stdResults[5].item())
print('F1 score: ', meanResults[6].item(), '(+/-)',stdResults[6].item())
print('Precision: ', meanResults[7].item(), '(+/-)',stdResults[7].item())
print('Recall: ', meanResults[8].item(), '(+/-)',stdResults[8].item())
print('Training time: ', meanResults[9].item(), '(+/-)',stdResults[9].item())
print('Testing time: ', meanResults[10].item(), '(+/-)',stdResults[10].item())

print('\n')
print('========== Network ==========')
print('Number of hidden layers: ', meanResults[11].item(), '(+/-)',stdResults[11].item())
print('Number of features: ', meanResults[12].item(), '(+/-)',stdResults[12].item())
print('Number of clusters: ', meanResults[13].item(), '(+/-)',stdResults[13].item())



========== Performance ==========
Preq Accuracy:  93.18 (+/-) 0.53
All tasks accuracy:  86.63 (+/-) 1.05
BWT:  -8.62 (+/-) 0.58
FWT:  18.65 (+/-) 3.64
ARI:  0.86 (+/-) 0.01
NMI:  0.86 (+/-) 0.01
F1 score:  0.93 (+/-) 0.01
Precision:  0.93 (+/-) 0.0
Recall:  0.93 (+/-) 0.01
Training time:  118.65 (+/-) 12.47
Testing time:  0.16 (+/-) 0.02


========== Network ==========
Number of hidden layers:  1.0 (+/-) 0.0
Number of features:  105.0 (+/-) 2.28
Number of clusters:  1206.0 (+/-) 39.68
